This file runs each step of pipeline seperately as a debug tool

In [41]:
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from ml_metadata.proto import metadata_store_pb2
from pipeline.e2e_pipeline.pipeline import create_pipeline

In [42]:
from pipeline.config import pipe_config

In [71]:
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = pipe_config.METADATA_PATH
connection_config.sqlite.connection_mode = 3
# Or
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    pipe_config.METADATA_PATH)

sqlite {
  filename_uri: "metadata/penguin-e2e/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

In [118]:
pipeline = create_pipeline(
        pipeline_name=pipe_config.PIPELINE_NAME,
        pipeline_root=pipe_config.PIPELINE_ROOT,
        data_root=pipe_config.DATA_ROOT,
        metadata_path=pipe_config.METADATA_PATH,
        schema_path=pipe_config.SAVED_SCHEMA_PATH,
        trainer_module_file=pipe_config.TRAINER_MODULE_PATH,
        serving_model_dir=pipe_config.SERVING_MODEL_DIR,
    )

In [119]:
pipeline.components[4]

Evaluator(spec: <tfx.types.standard_component_specs.EvaluatorSpec object at 0x16e125dc0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x16e125c40>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Evaluator, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'model': Channel(
    type_name: Model
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'baseline_model': Channel(
    type_name: Model
    artifacts: [Artifact(artifact: , artifact_type: name: "Model"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [97]:
# get lattest trainer artifacts
from pipeline.schema_pipeline.utils import get_latest_artifacts
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

with Metadata(metadata_connection_config) as metadata_handler:
    # Find output artifacts from MLMD.
    trainer_outputs = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                          'Trainer')
trainer_outputs

defaultdict(list,
            {'model': [Artifact(artifact: id: 24
              type_id: 12
              uri: "pipeline_output/penguin-e2e/Trainer/model/36"
              custom_properties {
                key: "name"
                value {
                  string_value: "penguin-e2e:2021-08-05T00:36:25.393680:Trainer:model:0"
                }
              }
              custom_properties {
                key: "tfx_version"
                value {
                  string_value: "1.0.0"
                }
              }
              state: LIVE
              create_time_since_epoch: 1628123841202
              last_update_time_since_epoch: 1628123841202
              , artifact_type: id: 12
              name: "Model"
              )],
             'model_run': [Artifact(artifact: id: 25
              type_id: 13
              uri: "pipeline_output/penguin-e2e/Trainer/model_run/36"
              custom_properties {
                key: "name"
                value {
         

In [115]:
from tfx.types import channel_utils

# get latest evaluator artifact
evaluator = pipeline.components[4]
# convert trainer Model artifact to Channel
evaluator.inputs['model'] = channel_utils.as_channel(trainer_outputs['model'])

In [116]:

evaluator

Evaluator(spec: <tfx.types.standard_component_specs.EvaluatorSpec object at 0x16e348760>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x16e384b50>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Evaluator, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'model': Channel(
    type_name: Model
    artifacts: [Artifact(artifact: id: 24
type_id: 12
uri: "pipeline_output/penguin-e2e/Trainer/model/36"
custom_properties {
  key: "name"
  value {
    string_value: "penguin-e2e:2021-08-05T00:36:25.393680:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.0.0"
  }
}
state: LIVE
create_time_since_epoch: 1628123841202
last_update_time_since_epoch: 1628123841202
, artifact_type: id: 12
name: "Model"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'baseline_model': Channel(
    type_name: Model
    artifacts: [Artifact(artifact: , artifact_type: name: "Model"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [117]:
context = InteractiveContext(pipeline_root=pipe_config.PIPELINE_ROOT, metadata_connection_config=connection_config)
context.run(evaluator)

ValueError: Unresolved input channel Artifact(artifact: , artifact_type: name: "Model"
) for input 'baseline_model' was passed in interactive mode. When running in interactive mode, upstream components must first be run with `interactive_context.run(component)` before their outputs can be used in downstream components.